### 프로젝트1
[과제 주제]
선 넘는 물체 인식

[구현 기능]
1. 웹캠 열기
2. 웹캠 화면 중앙에 선 출력
3. YOLO 적용하여 객체 탐지
4. 화면 가운데 선을 넘는 객체 추적하기
5. 선을 넘은 객체의 객체명별로 카운트하여 우측 상단에 라벨로 출력

[추가 기능]
1. 객체가 선에 닿아있으면 경고 알림
2. 선데 닿은 객체의 바운딩 박스 색을 빨간색으로 변경

In [2]:
# 라이브러리 불러오기
import cv2
import numpy as np
import matplotlib.pyplot as plt
import threading
from ultralytics import YOLO

In [3]:
model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')
model.info()

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(129, 3157200, 0, 8.8575488)

In [1]:
def draw_detections(image, results):

    for result in results:  # 결과 리스트를 반복
        for detection in result.boxes:  # 각 결과 내의 박스들을 반복

            x1, y1, x2, y2 = map(int, detection.xyxy[0])  # 박스의 좌표를 정수로 변환하여 추출
            conf = detection.conf[0]  # 검출된 객체의 신뢰도 추출

            class_id = int(detection.cls[0])  # 검출된 객체의 클래스 ID를 정수로 변환하여 추출
            label = f"{model.names[class_id]}: {conf:.2f}"  # 라벨을 클래스 이름과 신뢰도로 설정
            color = (0, 255, 0)  # 박스와 텍스트 색상을 녹색으로 설정

            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)  # 이미지에 사각형(박스) 그리기
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)  # 이미지에 텍스트(라벨) 추가
            
    return image  # 수정된 이미지를 반환


In [1]:
# 객체 카운터 딕셔너리 초기화
object_counts = {}

# 객체가 선을 넘었는지 확인하는 함수
def check_line_crossing(box, line_y):
    _, y1, _, y2 = map(int, box)
    return y1 <= line_y <= y2

def main():
    # 웹캠 열기
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("웹캠을 열 수 없습니다.")
        return

    # 웹캠 프레임 크기 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # 중앙선 y좌표
    center_line_y = height // 2

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO로 객체 감지
        results = model(frame)
        
        # 중앙선 그리기
        cv2.line(frame, (0, center_line_y), (width, center_line_y), 
                (255, 255, 0), 2)

        # 감지된 객체 처리
        for result in results[0]:
            boxes = result.boxes
            for box in boxes:
                # 바운딩 박스 좌표
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                # 객체 클래스 및 신뢰도
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                class_name = model.names[cls]

                # 선을 넘었는지 확인
                crossed = check_line_crossing(box.xyxy[0], center_line_y)
                
                # 박스 색상 설정 (선을 넘으면 빨간색)
                color = (0, 0, 255) if crossed else (0, 255, 0)
                
                # 바운딩 박스 그리기
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                
                # 객체 레이블 표시
                label = f"{class_name} {conf:.2f}"
                cv2.putText(frame, label, (x1, y1-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                
                # 선을 넘은 객체 카운트
                if crossed:
                    if class_name not in object_counts:
                        object_counts[class_name] = 1
                    else:
                        object_counts[class_name] += 1
                    
                    # 경고 메시지 출력
                    cv2.putText(frame, "WARNING: Object Crossing!", 
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                            1, (0, 0, 255), 2)

        # 카운트 정보 표시
        y_offset = 50
        for class_name, count in object_counts.items():
            text = f"{class_name}: {count}"
            cv2.putText(frame, text, (width-200, y_offset), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            y_offset += 30

        # 결과 화면 표시
        cv2.imshow('Object Detection', frame)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # 자원 해제
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


0: 480x640 1 person, 69.3ms
Speed: 1.4ms preprocess, 69.3ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 51.4ms
Speed: 2.0ms preprocess, 51.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 47.6ms
Speed: 2.2ms preprocess, 47.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 49.1ms
Speed: 1.2ms preprocess, 49.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 51.0ms
Speed: 1.2ms preprocess, 51.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 49.8ms
Speed: 1.5ms preprocess, 49.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 85.4ms
Speed: 1.2ms preprocess, 85.4ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 49.4ms
Speed: 1.2ms preprocess, 49.4ms inference, 1.0ms postprocess per image at shape (1, 3, 48